In [24]:
import requests 
from bs4 import BeautifulSoup 
from urllib.parse import urljoin

In [ ]:
# 회차 목록 크롤링

In [33]:

def 웹툰_회차_목록_가져오기(title_id, page=1):
    """
    지정된 title_id의 웹툰 목록에서 지정 page의 회차 목록을 리스트로 반환합니다.
    """
    # index_page_url = "https://m.comic.naver.com/webtoon/list?titleId=769209&page=1"  # 화산귀환
    # index_page_url = f"https://m.comic.naver.com/webtoon/list?titleId={title_id}&page={page}"         # 전독시

    index_page_url = "https://m.comic.naver.com/webtoon/list"
    params = {  # parameters
        "titleId": title_id,
        "page": page,
    }
    
    res = requests.get(index_page_url, params=params)
    html = res.text
    soup = BeautifulSoup(html, "html.parser")
    
    ep_tag_list = soup.select("[data-title-id]")
    ep_list = []  # ADDED
    
    for ep_tag in ep_tag_list:
        ep_title_id = ep_tag["data-title-id"]
        ep_no = ep_tag["data-no"]
        ep_title = ep_tag.select_one(".title").text.strip()
        ep_url = ep_tag.select_one("a")["href"]
        if ep_url != "#":
            ep_url = urljoin(index_page_url, ep_url)
            # print(ep_title, ep_url)
            ep_list.append({
                "title": ep_title,
                "url": ep_url,
                "title_id": ep_title_id,
                "no": ep_no,
            })  # ADDED
    
    # len(ep_list)  # ADDED

    return ep_list

In [26]:
화산귀환_아이디 = "747269"

웹툰_회차_목록_가져오기(화산귀환_아이디, 4)

[{'title': '072. Ep.15 왕이 없는 세계 (8)',
  'url': 'https://m.comic.naver.com/webtoon/detail?titleId=747269&no=73&week=wed&listSortOrder=DESC&listPage=4',
  'title_id': '747269',
  'no': '73'},
 {'title': '071. Ep.15 왕이 없는 세계 (7)',
  'url': 'https://m.comic.naver.com/webtoon/detail?titleId=747269&no=72&week=wed&listSortOrder=DESC&listPage=4',
  'title_id': '747269',
  'no': '72'},
 {'title': '070. Ep.15 왕이 없는 세계 (6)',
  'url': 'https://m.comic.naver.com/webtoon/detail?titleId=747269&no=71&week=wed&listSortOrder=DESC&listPage=4',
  'title_id': '747269',
  'no': '71'},
 {'title': '069. Ep.15 왕이 없는 세계 (5)',
  'url': 'https://m.comic.naver.com/webtoon/detail?titleId=747269&no=70&week=wed&listSortOrder=DESC&listPage=4',
  'title_id': '747269',
  'no': '70'},
 {'title': '068. Ep.15 왕이 없는 세계 (4)',
  'url': 'https://m.comic.naver.com/webtoon/detail?titleId=747269&no=69&week=wed&listSortOrder=DESC&listPage=4',
  'title_id': '747269',
  'no': '69'},
 {'title': '067. Ep.15 왕이 없는 세계 (3)',
  'url': 'ht

In [ ]:
# 웹툰 이미지 주소 리스트 뽑아내기

In [36]:
# url = "https://m.comic.naver.com/webtoon/detail?titleId=747269&no=163&week=wed&listSortOrder=DESC&listPage=1"

def 웹툰_특정회차_이미지_목록_가져오기(title_id, no) :
    url = "https://m.comic.naver.com/webtoon/detail"
    params = {
        "titleId" : title_id,
        "no" : no,
        "week" : "wed",
        "listSortOrder" : "DESC",
        "listPage" : 1,
    }
    res = requests.get(url, params=params)
    html = res.text
    soup = BeautifulSoup(html, "html.parser")
    image_url_list = []

    image_tag_list = soup.select(".toon_image")
    for image_tag in image_tag_list:
        image_url = image_tag["data-src"]
        image_url_list.append(image_url)
        
    return image_url_list

In [28]:
image_url_list = 웹툰_특정회차_이미지_목록_가져오기(화산귀환_아이디, 163)
len(image_url_list)

91

In [31]:
title_id = 747269
no = 163

import os
from tqdm.notebook import tqdm

def 웹툰_특정회차_이미지_목록_다운받기(base_path, image_url_list) :
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    }

    # 순차적으로 하나씩 요청해서 저장 : 멀티쓰레드, 멀티프로세스 등을 활용하면 좀 더 빠른 처리가 가능하다.

    for image_url in tqdm(image_url_list):
        res = requests.get(image_url, headers=headers)
        res.raise_for_status()  # 정상아니면 예외 발생

        image_filename = os.path.basename(image_url)  # 파일명 문자열만 뽑아줍니다.

        # filepath = f"a/b/c/"
        # filepath = os.path.join("a", "b", "c")abs
        
        filepath = os.path.join(base_path, image_filename)

        dirpath = os.path.dirname(filepath)
        os.makedirs(dirpath, exist_ok=True)

        image_data = res.content
        with open(filepath, "wb") as f:
            f.write(image_data)

In [32]:
title_id = 747269
no = 163

base_path = os.path.join("webtoons", str(title_id), str(no))
if not os.path.exists(base_path) :  
    웹툰_특정회차_이미지_목록_가져오기(base_path, image_url_list)
else :
    print(f"이미 다운받은 회차입니다. - {title_id}, {no}")

이미 다운받은 회차입니다. - 747269, 163


In [ ]:
# 종합

In [38]:
title_id = 747269 # 화산귀환


episode_list = 웹툰_회차_목록_가져오기(title_id)
print(len(episode_list))
episode_list[0]

30


{'title': '162. Ep. 28 최강의 희생양 (6)',
 'url': 'https://m.comic.naver.com/webtoon/detail?titleId=747269&no=163&week=wed&listSortOrder=DESC&listPage=1',
 'title_id': '747269',
 'no': '163'}

In [ ]:
for episode in episode_list :
    
    image_url_list = 웹툰_특정회차_이미지_목록_가져오기(episode["title_id"], episode["no"])
    
    base_path = os.path.join("webtoons", str(episode["title_id"]), str(episode["no"]))
    
    if not os.path.exists (base_path):
        print(f'{episode["title_id"], }")
        
    print(episode)
    break
    # 구현중에라서, 하나만 순회도록 합니다.